## 交互式 Phi 3 Mini 4K 指令聊天机器人与 Whisper

### 介绍：
交互式 Phi 3 Mini 4K 指令聊天机器人是一款工具，用户可以通过文本或语音输入与 Microsoft Phi 3 Mini 4K 指令演示进行互动。该聊天机器人可用于多种任务，例如翻译、天气更新以及一般信息查询。


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[创建你的 Huggingface 访问令牌](https://huggingface.co/settings/tokens)

创建一个新的令牌  
提供一个新的名称  
选择写入权限  
复制令牌并将其保存在安全的地方


以下是Python代码的主要功能：导入`os`模块并设置一个环境变量。

1. 导入`os`模块：
   - Python中的`os`模块提供了与操作系统交互的方式。它允许执行各种与操作系统相关的任务，例如访问环境变量、操作文件和目录等。
   - 在这段代码中，使用`import`语句导入了`os`模块。通过这个语句，可以在当前的Python脚本中使用`os`模块的功能。

2. 设置环境变量：
   - 环境变量是操作系统中可以被程序访问的值。它是一种存储配置设置或其他信息的方式，可以被多个程序使用。
   - 在这段代码中，通过`os.environ`字典设置了一个新的环境变量。字典的键是`'HF_TOKEN'`，值是从变量`HUGGINGFACE_TOKEN`中获取的。
   - `HUGGINGFACE_TOKEN`变量在代码片段的上方定义，并使用`#@param`语法赋值为字符串`"hf_**************"`。这种语法通常用于Jupyter笔记本中，允许用户直接在笔记本界面中输入参数或进行配置。
   - 设置了`'HF_TOKEN'`环境变量后，程序的其他部分或运行在同一操作系统上的其他程序都可以访问这个变量。

总体来说，这段代码导入了`os`模块，并将名为`'HF_TOKEN'`的环境变量设置为`HUGGINGFACE_TOKEN`变量中提供的值。


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

这个代码片段定义了一个名为 `clear_output` 的函数，用于清除 Jupyter Notebook 或 IPython 当前单元格的输出。让我们分解代码并理解其功能：

`clear_output` 函数接受一个名为 `wait` 的参数，它是一个布尔值。默认情况下，`wait` 设置为 `False`。这个参数决定了函数是否应该等待新的输出可用后再替换现有输出，然后清除它。

该函数的作用是清除当前单元格的输出。在 Jupyter Notebook 或 IPython 中，当一个单元格产生输出（例如打印的文本或图形化的绘图）时，这些输出会显示在单元格下方。`clear_output` 函数允许你清除这些输出。

代码片段中没有提供函数的具体实现，代码中用省略号（...）表示。这些省略号是实际执行清除输出操作的代码的占位符。函数的实现可能涉及与 Jupyter Notebook 或 IPython 的 API 交互，以移除单元格中的现有输出。

总体来说，这个函数提供了一种方便的方法来清除 Jupyter Notebook 或 IPython 当前单元格的输出，使得在交互式编码过程中更容易管理和更新显示的内容。


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

使用 Edge TTS 服务进行文本转语音 (TTS)。以下是相关函数的逐一实现：

1. `calculate_rate_string(input_value)`：此函数接收一个输入值并计算 TTS 语音的速率字符串。输入值表示所需的语速，其中值为 1 表示正常速度。函数通过从输入值中减去 1，再乘以 100，然后根据输入值是否大于或等于 1 来确定符号。函数返回格式为“{sign}{rate}”的速率字符串。

2. `make_chunks(input_text, language)`：此函数接收输入文本和语言作为参数。它根据语言的特定规则将输入文本拆分为多个块。在此实现中，如果语言是“English”，函数会在每个句号（“.”）处拆分文本，并去除任何前后空格。然后为每个块添加句号，并返回过滤后的块列表。

3. `tts_file_name(text)`：此函数根据输入文本生成 TTS 音频文件的文件名。它对文本进行多种转换：移除末尾的句号（如果存在），将文本转换为小写，去除前后空格，并将空格替换为下划线。然后将文本截断为最多 25 个字符（如果超过），或者使用完整文本（如果为空）。最后，它使用 [`uuid`] 模块生成一个随机字符串，并将其与截断后的文本组合，生成格式为“/content/edge_tts_voice/{truncated_text}_{random_string}.mp3”的文件名。

4. `merge_audio_files(audio_paths, output_path)`：此函数将多个音频文件合并为一个音频文件。它接收音频文件路径列表和输出路径作为参数。函数初始化一个空的 `AudioSegment` 对象 [`merged_audio`]。然后遍历每个音频文件路径，使用 `pydub` 库的 `AudioSegment.from_file()` 方法加载音频文件，并将当前音频文件追加到 [`merged_audio`] 对象中。最后，它将合并后的音频以 MP3 格式导出到指定的输出路径。

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`：此函数使用 Edge TTS 服务执行 TTS 操作。它接收文本块列表、语速、语音名称和保存路径作为参数。如果块的数量大于 1，函数会为存储单个块音频文件创建一个目录。然后遍历每个块，使用 `calculate_rate_string()` 函数、语音名称和块文本构建 Edge TTS 命令，并使用 `os.system()` 函数执行命令。如果命令执行成功，它会将生成的音频文件路径追加到列表中。在处理完所有块后，它使用 `merge_audio_files()` 函数合并单个音频文件，并将合并后的音频保存到指定的保存路径。如果只有一个块，它会直接生成 Edge TTS 命令并将音频保存到保存路径。最后，它返回生成的音频文件的保存路径。

6. `random_audio_name_generate()`：此函数使用 [`uuid`] 模块生成随机音频文件名。它生成一个随机 UUID，将其转换为字符串，取前 8 个字符，添加“.mp3”扩展名，并返回随机音频文件名。

7. `talk(input_text)`：此函数是执行 TTS 操作的主要入口。它接收输入文本作为参数。首先检查输入文本的长度，以确定是否为长句（长度大于或等于 600 个字符）。根据长度和 `translate_text_flag` 变量的值，它确定语言并使用 `make_chunks()` 函数生成文本块列表。然后使用 `random_audio_name_generate()` 函数生成音频文件的保存路径。最后，它调用 `edge_free_tts()` 函数执行 TTS 操作，并返回生成的音频文件的保存路径。

总体而言，这些函数协同工作，将输入文本拆分为多个块，生成音频文件的文件名，使用 Edge TTS 服务执行 TTS 操作，并将单个音频文件合并为一个音频文件。


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

实现了两个函数：convert_to_text 和 run_text_prompt，以及声明了两个类：str 和 Audio。

convert_to_text 函数以 audio_path 作为输入，使用名为 whisper_model 的模型将音频转录为文本。该函数首先检查 gpu 标志是否设置为 True。如果是，则使用 whisper_model，并设置一些参数，例如 word_timestamps=True、fp16=True、language='English' 和 task='translate'。如果 gpu 标志为 False，则使用 whisper_model 并设置 fp16=False。生成的转录结果会保存到名为 'scan.txt' 的文件中，并作为文本返回。

run_text_prompt 函数以 message 和 chat_history 作为输入。它使用 phi_demo 函数根据输入消息从聊天机器人生成响应。生成的响应会传递给 talk 函数，将响应转换为音频文件并返回文件路径。Audio 类用于显示和播放音频文件。音频通过 IPython.display 模块的 display 函数显示，并使用 autoplay=True 参数创建 Audio 对象，使音频自动播放。chat_history 会更新为输入消息和生成的响应，并返回一个空字符串和更新后的 chat_history。

str 类是 Python 中的内置类，表示字符序列。它提供了多种方法来操作和处理字符串，例如 capitalize、casefold、center、count、encode、endswith、expandtabs、find、format、index、isalnum、isalpha、isascii、isdecimal、isdigit、isidentifier、islower、isnumeric、isprintable、isspace、istitle、isupper、join、ljust、lower、lstrip、partition、replace、removeprefix、removesuffix、rfind、rindex、rjust、rpartition、rsplit、rstrip、split、splitlines、startswith、strip、swapcase、title、translate、upper、zfill 等。这些方法允许执行搜索、替换、格式化和操作字符串等操作。

Audio 类是一个自定义类，表示音频对象。它用于在 Jupyter Notebook 环境中创建音频播放器。该类接受多个参数，例如 data、filename、url、embed、rate、autoplay 和 normalize。data 参数可以是 numpy 数组、样本列表、表示文件名或 URL 的字符串，或原始 PCM 数据。filename 参数用于指定加载音频数据的本地文件，url 参数用于指定下载音频数据的 URL。embed 参数决定音频数据是使用数据 URI 嵌入还是引用原始来源。rate 参数指定音频数据的采样率。autoplay 参数决定音频是否自动播放。normalize 参数指定音频数据是否应归一化（重新调整到最大可能范围）。Audio 类还提供了 reload 方法，用于从文件或 URL 重新加载音频数据，以及 src_attr、autoplay_attr 和 element_id_attr 等属性，用于检索 HTML 中音频元素的对应属性。

总体而言，这些函数和类用于将音频转录为文本，从聊天机器人生成音频响应，以及在 Jupyter Notebook 环境中显示和播放音频。


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
